# 專題（一）：訓練LSTM之歌詞自動填詞器

## 專案目標
- 目標：使用 LSTM 模型去學習五月天歌詞，並且可以自動填詞來產生歌詞
- mayday_lyrics.txt 資料說明：
    - 每一行都是一首歌的歌詞
    - 除去標點符號並以空白表示間格
- 利用 mayday_lyrics.txt 來產生歌詞的序列
- 使用 LSTM 模型去學習歌詞的序列
- 當我們給定開頭的一段歌詞，例如：”給我一首歌”，就可以用 LSTM 猜下一個字，反覆這個過程就可以自動填詞

## 實作提示
- STEP1：從 mayday_lyrics.txt 中取出歌詞
- STEP2：建立每個字的 Index
- STEP3：用 Rolling 的方式打造 LyricsDataset
- STEP4：使用 DataLoader 來包裝 LyricsDataset
- STEP5：建立 LSTM 模型： inputs > nn.Embedding > nn.LSTM > nn.Dropout > 取最後一個 state > nn.Linear > softmax
- STEP6：開始訓練並調整參數
- STEP7：進行 Demo，給定 pre_text ，使用模型迭代的預測下一個字產生歌詞
- (進階) STEP8：在 Demo 時可以採用依照 Softmax 機率來作隨機採樣，這可以增加隨機性，讓歌詞有更多變化，當然你還可以使用機率閥值來避免太奇怪的字出現

## 重要知識點：專題結束後你可以學會
- 如何讀取並處理需要 Rolling 的序列資料
- 了解如何用 Pytorch 建制一個 LSTM 的模型
- 學會如何訓練一個語言模型
- 學會如何隨機抽樣自 Softmax 的分布

In [1]:
# 連接個人資料 讀取 ＰＴＴ 訓練資料和儲存模型
#先連接自己的GOOGLE DRIVE 為了要儲存資料和訓練模型
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

# Current directory
print(os.getcwd())

# change directory
os.chdir('/content/drive/MyDrive/python_training/NLP100Days-part2/project_1_1/')
print(os.getcwd())

/content
/content/drive/MyDrive/python_training/NLP100Days-part2/project_1_1


In [3]:
!pip install torch
#!pip install transformers
# 設定 torchtext 版本 安裝完必須重新啟動執行階段
!pip install torchtext==0.6.0

In [4]:
import pandas as pd
import numpy as np

import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm
from collections import Counter

In [5]:
# from: https://github.com/gaussic/Chinese-Lyric-Corpus
with open('mayday_lyrics.txt', encoding='utf-8') as f:
    lyrics_list = [line.strip() for line in f.readlines()]

In [6]:
lyrics_list[:5]

['摸不到的顏色 是否叫彩虹 看不到的擁抱 是否叫做微風 一個人 想著一個人 是否就叫寂寞 命運偷走如果 只留下結果 時間偷走初衷 只留下了苦衷 你來過 然後你走後 只留下星空 那一年我們望著星空 有那麼多的燦爛的夢 以為快樂會永久 像不變星空 陪著我 獵戶 天狼 織女光年外沈默 回憶 青春 夢想何時偷偷隕落 我愛過 然後我沈默 人海裡漂流 那一年我們望著星空 未來的未來從沒想過 當故事失去美夢 美夢失去線索 而我們失去聯絡 這一片無言無語星空 為什麼靜靜看我淚流 如果你在的時候 會不會伸手 擁抱我 細數繁星閃爍 細數此生奔波 原來所有 所得 所獲 不如一夜的星空 空氣中的溫柔 回憶你的笑容 徬佛只要伸手 就能觸摸 摸不到的顏色 是否叫彩虹 看不到的擁抱 是否叫做微風 一個人 習慣一個人 這一刻獨自望著星空 從前的從前從沒變過 寂寞可以是忍受 也可以是享受 享受僅有的擁有 那一年我們望著星空 有那麼多的燦爛的夢 至少回憶會永久 像不變星空 陪著我 最後只剩下星空 像不變回憶 陪著我',
 '如果你眼神能夠為我 片刻的降臨 如果你能聽到 心碎的聲音 沈默的守護著你 沈默的等奇跡 沈默的讓自己 像是空氣 大家都吃著聊著笑著 今晚多開心 最角落里的我 笑得多合群 盤底的洋蔥像我 永遠是調味品 偷偷地看著你 偷偷地隱藏著自己 如果你願意一層一層一層的剝開我的心 你會發現你會訝異 你是我最壓抑最深處的秘密 如果你願意一層一層一層的剝開我的心 你會鼻酸你會流淚 只要你能聽到我看到我的全心全意 聽你說你和你的他們 曖昧的空氣 我和我的絕望 裝得很風趣 我就像一顆洋蔥 永遠是配角戲 多希望能與你 有一秒專屬的劇情 如果你願意一層一層一層的剝開我的心 你會發現你會訝異 你是我最壓抑最深處的秘密 如果你願意一層一層一層的剝開我的心 你會鼻酸你會流淚 只要你能聽到我看到我的全心全意 如果你願意一層一層一層的剝開我的心 你會發現你會訝異 你是我最壓抑最深處的秘密 如果你願意一層一層一層的剝開我的心 你會鼻酸你會流淚 只要你能看到我聽到我的全心全意 你會鼻酸你會流淚 只要你能聽到我看到我的全心全意',
 '人群中哭著 你只想變成透明的顏色 你再也不會夢或痛或心動了 你已經決定了 你已經決定了 你靜靜忍著 緊緊把昨天在拳心握著 而回憶越是甜就是越傷人 越是在手心留下 密密麻麻深深淺淺的刀割 你

In [7]:
# 建立詞典對照表
from collections import Counter
cnt = Counter(''.join(lyrics_list))
word2index = {word: idx for idx, word in enumerate(cnt)}
index2word = {idx: word for word, idx in word2index.items()}

In [8]:
cnt[' ']

8587

In [9]:
cnt['摸']

10

In [10]:
vocab_siz=len(word2index)
vocab_siz


2101

In [11]:
# 建立數據集
class LyricsDataset(Dataset):
    def __init__(self, lyrics_list, word2index, num_unrollings=10):
      ## Code Here
        self.word2index = word2index
        self.samples = []
        for lyrics in lyrics_list:
            for idx in range(len(lyrics) - num_unrollings + 1):##重覆取num_unrollings為一組數據
                self.samples.append(lyrics[idx:idx + num_unrollings])

    def __getitem__(self, idx):
      ## Code Here
        sample = self.samples[idx]
        #取前num_unrollings-1為training data
        input_lyric = torch.LongTensor([self.word2index[w] for w in sample[:-1]])
        #取最後一字為label
        output_lyric = torch.LongTensor([self.word2index[sample[-1]]])

        return input_lyric, output_lyric

    def __len__(self):
        return len(self.samples)

In [12]:
batch_size = 128

dataset = LyricsDataset(lyrics_list, word2index)

train_loader = DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=True)

In [13]:
class LSTM_LM(nn.Module):
    def __init__(self, vocab_size, n_hidden, num_layers, dropout_ratio):
        super(LSTM_LM, self).__init__()
        # Code Here
        self.embedded = nn.Embedding(vocab_size, n_hidden)
        self.lstm = nn.LSTM(input_size=n_hidden,
                            hidden_size=n_hidden,
                            num_layers=num_layers,
                            batch_first=True,
                            dropout=dropout_ratio)
        self.fc = nn.Linear(n_hidden, vocab_size)
        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, inputs):
      ## Code Here
        embedded = self.embedded(inputs)  # [batch_size, num_unrollings - 1, n_hidden]
        outputs, _ = self.lstm(embedded)
        outputs = self.dropout(outputs)
        outputs = outputs[:,-1]  # [batch_size, n_hidden]
        logits = self.fc(outputs)# [batch_size, vocab_size]

        return logits

In [14]:
# 載入 pytorch 套件, 依照現有環境判定是否使用 GPU 計算
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [15]:
def train_batch(model, data, criterion, optimizer, device):##
    model.train()
    inputs, targets = [d.to(device) for d in data]
    outputs = model(inputs)
    loss = criterion(outputs, targets.view(-1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()

In [16]:
# 訓練模型
epochs = 100
lr = 0.001

model = LSTM_LM(vocab_siz,128, 2, 0.2) ##128, len(word2index), 2, 0.2

model.to(device)

criterion = nn.CrossEntropyLoss(size_average=False)
#criterion.to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)


for epoch in range(1, 1 + epochs):
    tot_train_loss = 0
    tot_train_count = 0

    for train_data in train_loader:
        loss = train_batch(model, train_data, criterion, optimizer, device)

        tot_train_loss += loss
        tot_train_count += train_data[0].size(0)

    print('epoch ', epoch, 'train_loss: ', tot_train_loss / tot_train_count)

    if epoch % 10 == 0:
        for idx in [0, 50, 99]:
            input_batch = dataset[idx][0].unsqueeze(0).to(device)
            predict = model(input_batch).argmax(dim=-1).item()
            print('Example: "{}"+"{}"'.format(dataset.samples[idx][:-1], index2word[predict]))

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


epoch  1 train_loss:  5.6476752105846755
epoch  2 train_loss:  5.210829824514929
epoch  3 train_loss:  4.881647736190876
epoch  4 train_loss:  4.613010041510349
epoch  5 train_loss:  4.377096993544495
epoch  6 train_loss:  4.160727202978758
epoch  7 train_loss:  3.9573896324851416
epoch  8 train_loss:  3.762858517385818
epoch  9 train_loss:  3.5894943487493576
epoch  10 train_loss:  3.4255059252598077
Example: "摸不到的顏色 是否"+"不"
Example: " 只留下結果 時間"+"一"
Example: "麼多的燦爛的夢 以"+"為"
epoch  11 train_loss:  3.2595346208916407
epoch  12 train_loss:  3.1206116621749658
epoch  13 train_loss:  2.978181263248158
epoch  14 train_loss:  2.8473125902610583
epoch  15 train_loss:  2.7252000885012433
epoch  16 train_loss:  2.6203404478189083
epoch  17 train_loss:  2.5131222139423883
epoch  18 train_loss:  2.4214995497727934
epoch  19 train_loss:  2.3179937237709782
epoch  20 train_loss:  2.231065293742337
Example: "摸不到的顏色 是否"+"就"
Example: " 只留下結果 時間"+"之"
Example: "麼多的燦爛的夢 以"+"為"
epoch  21 train_loss:  2.14

In [17]:
# 模型inference
pre_text = '給我一首歌'
generate_len = 50
prob_threshold = 0.01

result = [word2index[c] for c in pre_text]
for _ in range(generate_len):
    input_example = torch.LongTensor([result]).to(device)
    logit = model(input_example)
    ## Code Here
    prob = F.softmax(logit, dim=-1)
    probs = torch.where(prob > prob_threshold, prob, torch.zeros_like(prob))
    predict = torch.multinomial(probs, 1).item()
    ## End
    result += [predict]

print(''.join([index2word[i] for i in result]))

給我一首歌 為衝淡了名代定降落 我心臟 能忘你 明天真正講不是不知道 這到夏天全整個故事 盛起 親像了雪如而采


In [18]:
logit

tensor([[-18.0657, -10.5613, -17.3867,  ..., -19.5275, -30.4466, -13.6933]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [19]:
prob

tensor([[6.5773e-10, 1.1944e-06, 1.2969e-09,  ..., 1.5246e-10, 2.7610e-15,
         5.2112e-08]], device='cuda:0', grad_fn=<SoftmaxBackward>)

In [20]:
probs

tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0',
       grad_fn=<SWhereBackward>)

In [21]:
predict

851